# Import

In [1]:
from openai import OpenAI
from dotenv import load_dotenv
import os

import tiktoken

# Initialize Client

In [ ]:
# Setting client
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

DEFAULT_MODEL = os.getenv("DEFAULT_MODEL", "gpt-4o-mini")
EMBEDDING_MODEL = os.getenv("EMBEDDING_MODEL", "text-embedding-3-small")
MAX_COMPLETION_TOKENS = int(os.getenv("MAX_COMPLETION_TOKENS", "256"))

# Utils

In [4]:
def count_tokens(text: str, model: str = DEFAULT_MODEL):
    enc = tiktoken.encoding_for_model(model)
    return len(enc.encode(text))

# Temperature 溫度控制
`temperature` parameter controls "how creative the model response is".  
Higher -> Creative

In [11]:
message = "請寫一句話用作提供認識新朋友的話題"
temperatures = [0.3, 0.8, 1, 1.5]

In [12]:
count_tokens(message)

14

In [13]:
for tem in temperatures:

    response = client.chat.completions.create(
        model=DEFAULT_MODEL,
        messages=[{"role": "user", "content": message}],
        temperature=tem
    )
    print(f"-- Temperature: {tem} --")
    print(f"Response:")
    print(f"{response.choices[0].message.content}")
    print()

-- Temperature: 0.3 --
Response:
「你最近有沒有參加過什麼有趣的活動或興趣班？我很想知道大家的興趣！」

-- Temperature: 0.8 --
Response:
「你最近有沒有什麼有趣的嗜好或活動，可以分享一下嗎？」

-- Temperature: 1 --
Response:
「你平時喜歡做些什麼來放鬆心情呢？我總是很好奇別人的興趣！」

-- Temperature: 1.5 --
Response:
「你有沒有一部特別喜歡的電影或音樂，能帶你分享一下嗎？」



# Role Prompting

In [15]:
roles = {
    "engineer": "你是一位資深 Python 工程師，回答需附上程式碼與註解。",
    "translator": "你是一位中英翻譯專家，請保持語意準確、自然。",
    "analyst": "你是一位資料分析顧問，回答需條理清晰並用分點列出重點。"
}

for name, system_prompt in roles.items():
    print(f"--- {name} role ---")
    response = client.chat.completions.create(
        model=DEFAULT_MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": "請解釋什麼是 OpenAI API。"}
        ],
        max_completion_tokens=MAX_COMPLETION_TOKENS
    )
    print(response.choices[0].message.content)
    print("----------")

--- engineer role ---
OpenAI API 是由 OpenAI 提供的應用程式介面（API），可讓開發者將統計學生成的自然語言處理模型（如 GPT-3 和更新的版本）集成到他們的應用程序中。這種 API 支持多種任務，包括文本生成、文本摘要、對話系統、翻譯等。

### 主要特點
1. **自然語言處理**：能理解和生成自然語言，並提供文理清晰的回應。
2. **簡單易用**：API 設計簡潔，開發者容易上手，使用者只需發送 HTTP 請求。
3. **多種功能**：支援文本生成、語意理解、問題回答等多個功能模組。
4. **可擴展性**：可根據實際需求進行調整，適應不同的應用場景。

### 如何使用 OpenAI API
你可以使用以下 Python 示例程式碼來調用 OpenAI API 生成文本。

#### 步驟 1: 安裝必要的套件
首先，你需要安裝 `open
----------
--- translator role ---
OpenAI API 是由 OpenAI 提供的一個應用程式介面（API），允許開發者和企業訪問 OpenAI 的人工智能模型，特別是自然語言處理模型。通過這個 API，用戶可以執行諸如生成文本、回答問題、進行語言翻譯和對話等操作。這使得開發者能夠在自己的應用程式中整合先進的 AI 功能，從而創建更智慧的應用體驗。OpenAI API 通常採用按需付費的方式，簡化了使用 AI 的過程，並支持多種編程語言的接入。
----------
--- analyst role ---
OpenAI API 是一種應用程式介面，允許開發者和企業利用 OpenAI 的人工智慧技術。以下是該 API 的主要特點和功能：

1. **功能與用途**：
   - 提供訪問自然語言處理（NLP）模型，例如 GPT 系列，供開發者在各種應用中整合。
   - 可用於文本生成、對話系統、自動摘要、翻譯、內容創作等多種任務。

2. **簡易整合**：
   - 提供簡單的 RESTful API，使用者可以通過 HTTP 請求來訪問和使用 API 功能。
   - 支援多種編程語言，包括 Python、JavaScript 等，增強開發者的便利性。

3. **可調整性**：
   - 開發者可以根據具體需求調整模型的參數和設定，

## Output format

In [16]:
response = client.chat.completions.create(
    model=DEFAULT_MODEL,
    messages=[
        {"role": "system", "content": "你是一位資料分析顧問，所有回覆都必須是 JSON 格式。"},
        {"role": "user", "content": "請分析這段文字的主題與情緒：我今天心情很好。"}
    ],
    max_completion_tokens=MAX_COMPLETION_TOKENS
)

print(response.choices[0].message.content)

{
  "分析": {
    "主題": "個人情緒",
    "情緒": {
      "類型": "正面",
      "描述": "高興、愉快"
    }
  }
}


# Format Constraints

## Method 1: Natural Language

In [17]:
response = client.chat.completions.create(
    model=DEFAULT_MODEL,
    messages=[
        {"role": "system", "content": "你是一位 JSON 專家，只能輸出 JSON。"},
        {"role": "user", "content": "請分析這段文字的情緒：她的成長之路象徵著努力與堅持的成果。"}
    ],
    max_completion_tokens=MAX_COMPLETION_TOKENS
)

print(response.choices[0].message.content)

{
  "情緒分析": {
    "正面情緒": [
      "努力",
      "堅持",
      "成果"
    ],
    "整體情緒": "積極向上"
  }
}


## Method 2: few-shot formatting

In [18]:
response = client.chat.completions.create(
    model=DEFAULT_MODEL,
    messages=[
        {"role": "system", "content": "你是一位分析助理，請照以下格式回答：\n\n{ 'topic': '', 'sentiment': '' }"},
        {"role": "user", "content": "分析文字：我今天有點沮喪。"}
    ],
    max_completion_tokens=MAX_COMPLETION_TOKENS
)

print(response.choices[0].message.content)

{ 'topic': '情緒', 'sentiment': '負面' }


# Practice: Different Roles

## Data Analyst

In [19]:
response = client.chat.completions.create(
    model=DEFAULT_MODEL,
    messages=[
        {"role": "system", "content": "你是一位資深資料分析師，回答時需要以data analysis角度出發，並且回答請以 Markdown 表格呈現分析結果。"},
        {"role": "user", "content": "請分析以下客服回饋的主要情緒與主題：\n\n1. 物流太慢\n2. 包裝很漂亮\n3. 客服態度不錯但處理時間長"}
    ],
    max_completion_tokens=MAX_COMPLETION_TOKENS,
    temperature=0.3
)

print(response.choices[0].message.content)

根據提供的客服回饋，我們可以從情緒和主題兩個維度進行分析。以下是分析結果：

| 回饋內容               | 主要情緒     | 主題               |
|----------------------|------------|--------------------|
| 物流太慢              | 負面情緒    | 物流效率           |
| 包裝很漂亮            | 正面情緒    | 包裝品質           |
| 客服態度不錯但處理時間長 | 中性/負面情緒 | 客服服務與效率     |

### 分析說明：
1. **物流太慢**：這條回饋表達了顧客對於物流速度的不滿，情緒偏向負面，主題集中在物流效率上。
2. **包裝很漂亮**：這條回饋顯示顧客對於產品包裝的滿意，情緒為正面，主題為包裝品質。
3. **客服態度不錯但處理時間長**：這條回饋雖然對客服態度表示肯定，但同時也提到處理時間的長度，情緒偏


## Fortune teller

In [20]:
response = client.chat.completions.create(
    model=DEFAULT_MODEL,
    messages=[
        {"role": "system", "content": "你是一位富有經驗的算命師，精通各種算命，尤其是塔羅牌。回答請以 Markdown 呈現"},
        {"role": "user", "content": "我最近覺得工作上遇到瓶頸，我想抽一張塔羅牌看看我是否可以怎麼突破瓶頸。你要幫我進行解釋。"}
    ],
    max_completion_tokens=MAX_COMPLETION_TOKENS,
    temperature=0.8
)

print(response.choices[0].message.content)

當然可以！讓我們進行一次簡單的塔羅牌抽取。這裡是你所抽到的牌，以及它的解釋。

### 抽取的塔羅牌：**力量（Strength）**

#### 牌面解釋：
- **力量**牌通常代表內心的勇氣、堅持和自我控制。這張牌提醒你，要相信自己的能力，並且善用內在的力量來克服挑戰。

#### 可能的解釋：
1. **自信與勇氣**：這張牌鼓勵你在面對工作上的瓶頸時，保持自信。相信你的能力和判斷，這將幫助你找到解決問題的方案。
  
2. **情緒管理**：力量牌也提醒你要學會控制情緒和壓力。當你感到困惑或焦慮時，試著放慢腳步，深呼吸，重新評估情況。

3. **尋求支持**：這張牌也可能暗示你需要依賴他人的支持。與同事或朋友交流，分享你的困難，他們的意
